Web scraping -> Celulares novos da https://www.kabum.com.br/

Imports

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import time
import re
driver = webdriver.Firefox()

Arrays auxiliares

In [ ]:
links_celulares = []
lista_pages_cell = []

marca = []
modelo = []
sistema_operacional = []
memoria_interna = []
memoria_ram = []
rede = []
resol_cam_tras = []
garantia = []
peso = []
preco = []

Função que captura todos os links de celulares novos do site da kabum e guarda no arquivo links.txt

In [ ]:
def pega_links():
    # Pegando todos as página onde tem os links de celulares
    for j in range(1, 28):
        lista_pages_cell.append(str("https://www.kabum.com.br/celular-smartphone?page_number={}&page_size=20&facet_filters=eyJDb25kacOnw6NvIjpbIk5vdm8iXX0=&sort=most_searched").format(j))

    # Para cada página, coleta todos os links.
    for page in lista_pages_cell:
        driver.get(page)
        celulares = driver.find_elements(By.CLASS_NAME, "productLink")
        for celular in celulares:
            smartphone = celular.get_attribute('href')

            # link numero 87: celular sem caracteristicas
            # ele conseguiu ir ate o celular 98. Depois a kabum nao permite
            if smartphone != "https://www.kabum.com.br/produto/429414/smartphone-motorola-moto-g53-5g-128gb-4gb-ram-octa-core-camera-dupla-50mp-tela-de-6-5-rose-pawr0002br":
                links_celulares.append(smartphone)

    with open("links.txt", "w") as arquivo:
        for link in links_celulares:
            arquivo.write(link + "\n")

    print(len(links_celulares))

Função que recebe um link (celular) e captura todas as informações necessárias

In [ ]:

def pega_infos(link):
    driver.get(link)

    div_infos = driver.find_element(By.ID, "secaoInformacoesTecnicas")
    infos = div_infos.text.split("\n")


    try:
        preco_cell = driver.find_element(By.CLASS_NAME, "finalPrice").text
        if preco_cell:
            preco.append(preco_cell)
        else:
            preco.append("")
    except selenium.common.exceptions.NoSuchElementException:
        preco.append(999)


    print("\n")
    print("##############################################################")
    print("\n")

    encontrou_marca = False
    encontrou_modelo = False
    encontrou_memoriaintGB = False
    encontrou_mp = False
    encontrou_meses = False
    encontrou_grama = False
  

    
    # Sistema Operacional ok
    if "Apple" in div_infos.text:
        sistema_operacional.append("IOS")
    else:
        sistema_operacional.append("Android")

    # Rede ok
    if "5G" in div_infos.text:
        rede.append("5G")
    elif "4G" in div_infos.text:
        rede.append("4G")
    elif "3G" in div_infos.text:
        rede.append("3G")
    else:
        rede.append("4G")
    
    # Memória Interna ok
    if "512GB" in div_infos.text:
        memoria_interna.append(512)
    elif "256GB" in div_infos.text:
        memoria_interna.append(256)
    elif "128GB" in div_infos.text:
        memoria_interna.append(128)
    elif "64GB" in div_infos.text:
        memoria_interna.append(64)
    elif "32GB" in div_infos.text:
        memoria_interna.append(32)
    elif "16GB" in div_infos.text:
        memoria_interna.append(16)
    elif "8GB" in div_infos.text:
        memoria_interna.append(8)
    else:
        memoria_interna.append("")
        print("aqui deu ruim na memoria interna")

    # Memória RAM ok
    if encontrou_memoriaintGB == False and ("Memória RAM" in div_infos.text or "- Memória RAM" in div_infos.text or "-Memória RAM" in div_infos.text or "Capacidade de RAM" in div_infos.text or 
    "-Capacidade de RAM" in div_infos.text or "- Memória RAM: " in div_infos.text):
        encontrou_memoriaintGB = True
        if "12GB" in div_infos.text:
            memoria_ram.append(12)
        elif "8GB" in div_infos.text:
            memoria_ram.append(8)
        elif "6GB" in div_infos.text:
            memoria_ram.append(6)
        elif "4GB" in div_infos.text:
            memoria_ram.append(4)
        elif "3GB" in div_infos.text:
            memoria_ram.append(3)
        elif "2GB" in div_infos.text:
            memoria_ram.append(2)
        else: 
            memoria_ram.append("")
            print("aqui deu ruim na RAM (provavelmente é iphone)")

    # Garantia ok
    if encontrou_meses == False and("meses" in div_infos.text or "Meses" in div_infos.text):
        if "meses" in div_infos.text:
            padrao = r'(\d+)\s*meses'
            matches = re.findall(padrao, div_infos.text)
            garantia.append(matches[-1])
            encontrou_meses = True

        elif "Meses" in div_infos.text:
            padraoM = r'(\d+)\s*Meses'
            matchesM = re.findall(padraoM, div_infos.text)
            garantia.append(matchesM[-1])
    
    # Peso ok
    if "gramas" in div_infos.text or "Gramas" in div_infos.text or "g" in div_infos.text:
        if "gramas" in div_infos.text or "Gramas" in div_infos.text:
            padrao = r'(\d+)\s*gramas'
            matches = re.findall(padrao, div_infos.text)
            peso.append(matches[-1])
            encontrou_grama = True

        elif "g" in div_infos.text:
            padraog = r'(\d+)\s*g'
            matchesg = re.findall(padraog, div_infos.text)
            peso.append(matchesg[-1])


    for item in infos:
        
        # Marca ok
        if encontrou_marca == False and ("Marca" in item or "- Marca" in item or "-Marca" in item):
            marca.append(item[item.find(':')+1:])
            encontrou_marca = True
            
        
            
        # Modelo ok
        elif encontrou_modelo == False and ("Modelo" in item or "- Modelo" in item or "-Modelo" in item):
            modelo.append(item[item.find(':')+1:])
            encontrou_modelo = True
        
            

        # Resolução camera traseira ok 
        elif encontrou_mp == False and ("MP" in item or "mp" in item):
            # typeof(item) = "string"
            if "MP" in item:
                matches = re.findall(r'\b(\d{2})MP\b', item)
                if matches:
                    resol_cam_tras.append(matches[0])
                    encontrou_mp = True

            elif encontrou_mp == False and("mp" in item):
                matchesm = re.findall(r'\b(\d{2})mp\b', item)
                if matchesm:
                    resol_cam_tras.append(matchesm[0])
                    encontrou_mp = True


        # Capacidade da bateria
        elif "Capacidade da bateria" or "- Capacidade da bateria" in item:
            pass

        
    if encontrou_marca == False:
        marca.append("")
    if encontrou_modelo == False:
        modelo.append("")
    if encontrou_memoriaintGB == False:
        memoria_ram.append("")
    if encontrou_meses == False:
        garantia.append("")
    if encontrou_grama == False:
        peso.append("")
    if encontrou_mp == False:
        resol_cam_tras.append("")

Procedimento para ler o arquivo links.txt e iterar nos links para aplicar a função pega_infos()
Montagem do DataSet.

In [ ]:
pega_links()


count = 0

batch_size = 10
output_file = "resultado_parcial.csv"

with open("links.txt", "r") as arquivo:
    links = arquivo.readlines()

for i in range(0, len(links), batch_size):
    batch_links = links[i:i + batch_size]

    data = {
        "marca": [],
        "modelo": [],
        "sistema_operacional": [],
        "rede": [],
        "memoria_interna": [],
        "memoria_ram": [],
        "resol_cam_tras": [],
        "garantia": [],
        "peso": [],
        "preco": []
    }

    for link in batch_links:
        pega_infos(link)
        count+=1
        print(count)

        data["marca"].append(marca[-1])
        data["modelo"].append(modelo[-1])
        data["sistema_operacional"].append(sistema_operacional[-1])
        data["rede"].append(rede[-1])
        data["memoria_interna"].append(memoria_interna[-1])
        data["memoria_ram"].append(memoria_ram[-1])
        data["resol_cam_tras"].append(resol_cam_tras[-1])
        data["garantia"].append(garantia[-1])
        data["peso"].append(peso[-1])
        data["preco"].append(preco[-1])

    print(len(marca))
    print(len(modelo))
    print(len(sistema_operacional))
    print(len(rede))
    print(len(memoria_interna))
    print(len(memoria_ram))
    print(len(resol_cam_tras))
    print(len(garantia))
    print(len(peso))
    print(len(preco))

    df = pd.DataFrame(data)
    df.to_csv(output_file, mode="a", header=not i)  # Salvar o DataFrame parcial no arquivo CSV
